In [488]:
import datetime
import lxml.html
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [489]:
# construção da URL
# datas
year = datetime.date.today().year
month = datetime.date.today().month
day = datetime.date.today().day
data_inicial = (f'{day}/{month}/{year-10}')
data_final = (f'{day}/{month}/{year}')

# ativo
ativo = 'PETR4'

#url
response = requests.get(f"https://www.infomoney.com.br/Pages/Download/Download.aspx?dtIni={data_inicial}&dtFinish={data_final}&Ativo={ativo}&Semana=null&Per=null&type=2&Stock={ativo}&StockType=1")
print("get page: ", response)

get page:  <Response [200]>


In [490]:
# criação do parser para navegação na árvore DOM
content = BeautifulSoup(url.content, "html.parser")
table = content.find_all("tr")
#(HTML(str(table)))

In [491]:
# Removing Multple spaces
def remove_multiple_spaces(string):
    """
    Se é String, então faz remoção de espaços em branco 
    replace: / por espaço em branco
    replace: , por . para facilitar a conversão de tipos
    """
    if type(string)==str:
        string_n = ' '.join(string.split()).replace('/',' ').replace(',','.')
        string = string_n.replace('Histórico', '-1').replace('Fech.', '-1')
        return string
    return string

In [492]:
# neste dataframe vou armazenar a data e o fechamento diário
df_petr4 = pd.DataFrame(columns=['Ano', 'Mes', 'Fechamento'])

# percorre table
for row in table:
    text = row.text
    text = remove_multiple_spaces(text)
    
    # insere dados diferentes a cada linha
    text_dados = text.split(sep=" ")
    
    # convert list to String
    ano = ''.join(text_dados[2:3])
    mes = ''.join(text_dados[1:2])
    fechamento = ''.join(text_dados[3:4])

    # variável p organizar a inserção no dataframe
    dados = pd.DataFrame([[ano, mes, fechamento]], columns=['Ano', 'Mes', 'Fechamento'])    
    df_petr4 = df_petr4.append(dados)

In [493]:
# remove rows with Strings
df_petr4 = df_petr4[1:]

,Ano,Mes,Fechamento
0,2018,10,26.32
0,2018,10,25.68
0,2018,10,26.20
0,2018,10,26.53
0,2018,10,25.92
0,2018,10,25.70
0,2018,10,26.45
0,2018,10,26.73
0,2018,10,25.77
0,2018,10,25.29


In [494]:
'''
# granularidade mensal
df_petr4 = df_petr4[df_petr4.Ano < '2018']

for ano in df_petr4.Ano: 
    print(ano)
    
    # percorre mês
    for mes in df_petr4.Mes:
        print(mes)
        df_petr4.mes = df_petr4.Fechamento
        
'''

"\n# granularidade mensal\ndf_petr4 = df_petr4[df_petr4.Ano < '2018']\n\nfor ano in df_petr4.Ano: \n    print(ano)\n    \n    # percorre mês\n    for mes in df_petr4.Mes:\n        print(mes)\n        df_petr4.mes = df_petr4.Fechamento\n        \n"

In [495]:
# Conversão de tipos object
df_petr4.Ano = df_petr4.Ano.astype('int16') # int8 não aceitou
df_petr4.Mes = df_petr4.Mes.astype('int8')
df_petr4.Fechamento = pd.to_numeric(df_petr4.Fechamento, downcast='float', errors='coerence')

print(df_petr4.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4963 entries, 0 to 0
Data columns (total 3 columns):
Ano           4963 non-null int16
Mes           4963 non-null int8
Fechamento    4962 non-null float32
dtypes: float32(1), int16(1), int8(1)
memory usage: 72.7 KB
None


### Salvamento do dataframe

In [496]:
df_petr4.to_csv('df_petr4.csv', 
                    index=False, sep=',', encoding='utf-8', decimal='.')
print('\n(df_petr4.csv) salvo!')

(df_petr4.csv) salvo!
